In [1]:
import keras.backend as K
from keras.models import load_model
from keras.layers import *
from keras import Model
import pickle
import imageio
import cv2

def crop_img(img):
    size = 256,256
    width, height = img.shape[:2]
    if img.ndim != 3: # remove bad images
        return None
    if width > height: # crop the image *on width
        center = width // 2
        left = center - height//2
        right = center + height//2
        img = img[left:right,:,:]
    else: # crop the image on height
        center = height // 2
        up = center - width//2
        down = center + width//2
        img = img[:,up:down,:]
    # resize the image
    img = cv2.resize(img, dsize=size)
    return img

def load_jpg(filepath):
    with open(filepath, 'rb') as f:
        img = np.array(imageio.imread(f), dtype=np.float32) / 255
    return crop_img(img).reshape(1,256,256,3)

def load(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)
    

dataTriple = load('obj/triplets.pkl')
triplets = dataTriple['triplets'] / 255
triplets = triplets[::4,0,:,:,:]

/home/henrik/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_darknet():
    model = load_model('obj/darknet.h5')
    for layer in model.layers:
        layer.trainable = False
    return model


def make_style_model():
    darknet = get_darknet()

    # pretrained layers
    inputs = darknet.layers[0].input
    xs = [darknet.get_layer('leaky_re_lu_%d'%d).output for d in range(3,7)]
    # Embedding model produces style embeddings
    base_model = Model(inputs, xs)
    
    # Crate triple model to train the base model
    # inputs for anchor, positive image, and negative image
    input_1 = Input((256, 256, 3))
    input_2 = Input((256, 256, 3))
    
    fs1 = base_model(input_1)
    fs2 = base_model(input_2)
    fs2 = [AveragePooling2D()(f2) for f2 in fs2]
    
    sty_dist = [Lambda(channel_dist)([f1, f2]) for f1,f2 in zip(fs1, fs2)]
    sty_dist = Add()(sty_dist)
    style_model = Model([input_1, input_2], sty_dist)
    
    return style_model

def channel_dist(x):
    x, y = x
    
    n1, _, n3 = map(int, x.shape[1:])
    m1, _, m3 = map(int, y.shape[1:])
    trans_vectors = K.reshape(x, (-1, n1**2, 1, n3)) - K.reshape(y, (-1, 1, m1**2, m3))
    eucl_dist = K.sum(K.square(trans_vectors), axis=3)
    poly_dist = K.min(eucl_dist, axis=1)
    return K.sum(poly_dist, axis=1, keepdims=True)
    

style_model = make_style_model()

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
input_1 = K.placeholder((None,256,256,3))
input_2 = K.placeholder((None,256,256,3))
fun = K.function([input_1, input_2], [style_model([input_1, input_2])])

In [4]:
im1= triplets[120].reshape(1,256,256,3)
im2 = load_jpg('obj/Starry-Night.jpg')
res = fun([im1, im2])
print(res)

[array([[109953.805]], dtype=float32)]


In [5]:
style_model.save('obj/style_model.h5')